In [1]:
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# json_file = 'datasets/test_data.json'
json_file = 'datasets/dataset.json'
with open(json_file,'r') as f:
    data = json.load(f)

In [3]:
df = pd.DataFrame(data, columns=['text', 'hasBadWords'])
df.rename(columns={"hasBadWords": "label"}, inplace=True)
df.label = df.label.apply(lambda x: 'spam' if x == True else 'ham')
# df['text'] = df['text'].apply(lambda x: str(x).split(' '))
# df = df[:20000]
df.head()
print(df.shape)

(86439, 2)


In [4]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Add the TextCategorizer to the empty model
textcat = nlp.add_pipe("textcat")

2023-03-01 11:34:07.604213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Add labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

In [6]:
train_texts = df['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in df['label']]

In [7]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('My Favorite Slut', {'cats': {'ham': True, 'spam': False}}),
 ("girlfriends sit on each other's faces with their asses",
  {'cats': {'ham': True, 'spam': False}}),
 ('bound beauty kisses her girlfriend',
  {'cats': {'ham': True, 'spam': False}})]

In [8]:
from spacy.util import minibatch
from spacy.training.example import Example

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) 
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

In [9]:
import random

random.seed(42)
spacy.util.fix_random_seed(42)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(1): #10
    random.shuffle(train_data)
    
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    
    # Iterate through minibatches
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 2977.051650292923}


In [10]:
texts = ["bastinado , feet , foot , sole , soles , toe , toe , toe job , foot job footjob , foot torture , foot torment , falaka , tickle , tickling , oiled feet , hose , hosiery , pantyhose , stockings , shoes , high heels , pumps , high heeled sandals , barefoot , toe torture , bound feet , bound foot , bound toes , heels , heel , instep , arches , arch , high arches , switching , cane , caning , crop , cropping , electrical shock , hairdryer , hot foot , hotfoot , gravel walk , breasts , ass , big ass , ass shaking , fire , foot whipping , hot pavement walks , snow walks , dirty feet , dirty foot , downtown barefoot walks .",
         "ass, anal, 18 & 19 yrs old, toys,butt, latina,blondes,ass spreading" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[0.99704236 0.00295759]
 [0.9971174  0.00288262]]


In [11]:
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['ham', 'ham']
